In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.preprocessing import LabelEncoder

In [ ]:
#data ingesation
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
#cleaning the data
df.drop('id', axis=1,inplace=True)

In [ ]:
df.head(2)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN


In [ ]:
df.drop('Unnamed: 32', axis=1, inplace=True)

In [ ]:
df.isnull().sum()

,0
diagnosis,0
radius_mean,0
texture_mean,0
perimeter_mean,0
area_mean,0
smoothness_mean,0
compactness_mean,0
concavity_mean,0
concave points_mean,0
symmetry_mean,0


In [ ]:
X = df.drop('diagnosis', axis=1)
y = df['diagnosis']

In [ ]:
#scaling
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
#Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#target Column encoding
enc = LabelEncoder()
y_train_encoded = enc.fit_transform(y_train)
y_test_encoded = enc.transform(y_test)

In [ ]:
#now we need to transoform numpy array into pytorch tensors
X_train_tensor = torch.from_numpy(X_train_scaled)
X_test_tensor = torch.from_numpy(X_test_scaled)
y_train_tensor = torch.from_numpy(y_train_encoded)
y_test_tensor = torch.from_numpy(y_test_encoded)

In [16]:
#defining our own model
class NNet():

  #first initialize the weights and biases using the constructor
  def __init__(self,X):
    #weights
    self.weights = torch.rand(X.shape[1],1, dtype=torch.float64, requires_grad=True)

    #biases
    self.bias = torch.zeros(1,dtype=torch.float64, requires_grad=True)

  def Forward_prop(self,X):
    #impose the formulation of wx+b
    z = torch.matmul(X,self.weights) + self.bias
    #now calculating the Y hat
    y_pred = torch.sigmoid(z)

    return y_pred

  def loss_function(self,y_pred,y):
    epsilon = 1e-7
    #Avoiding log(negative)
    y_pred = torch.clamp(y_pred,epsilon,1- epsilon)


    # Calculate loss
    loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
    return loss

In [17]:
#assigning the parameters
learning_rate = 0.1
epochs = 20

In [18]:
#training pipeline

#initialized model
model = NNet(X_train_tensor)

In [20]:
# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model.Forward_prop(X_train_tensor)

  # loss calculate
  loss = model.loss_function(y_pred, y_train_tensor)

  # backward pass
  loss.backward()

  # parameters update
  with torch.no_grad():
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad

  # zero gradients
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 3.7681070267154197
Epoch: 2, Loss: 3.651543254726035
Epoch: 3, Loss: 3.530810052620373
Epoch: 4, Loss: 3.4026233903312737
Epoch: 5, Loss: 3.2704320869355814
Epoch: 6, Loss: 3.136742459298743
Epoch: 7, Loss: 3.0050097852469513
Epoch: 8, Loss: 2.8722461878935484
Epoch: 9, Loss: 2.731088124053298
Epoch: 10, Loss: 2.5916222964216105
Epoch: 11, Loss: 2.4552099317797023
Epoch: 12, Loss: 2.3182159939297424
Epoch: 13, Loss: 2.1822670557658177
Epoch: 14, Loss: 2.0493934043260458
Epoch: 15, Loss: 1.9152205808903833
Epoch: 16, Loss: 1.7873301623727442
Epoch: 17, Loss: 1.663501343462225
Epoch: 18, Loss: 1.549791157373887
Epoch: 19, Loss: 1.4458077171361883
Epoch: 20, Loss: 1.3514008665952126


In [23]:
# model evaluation
with torch.no_grad():
  y_pred = model.Forward_prop(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5775623321533203
